In [ ]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# curl -o input.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

zsh:1: command not found: wget


In [2]:
# read it in to inspect it 
with open('input.txt', 'r') as file:
    text = file.read()

In [3]:
print("length of dataset is: ", len(text))

length of dataset is:  1115394


In [4]:
# lets look at first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# here all the unique characters in the file
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print('vocab size: ', vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size:  65


In [6]:
# create a dictionary mapping characters to integers
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()} # inverse
encode = lambda s: [stoi[c] for c in s] # encoder: Take in string, return list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: Take in list of integers, return string

print(encode('hi there'))
print(decode(encode('hi there')))
 # Google use SentencePiece for tokenization (subword tokenization)
 # openai uses tiktoken, a byte pair encoding algorithm tokenization

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [7]:
import tiktoken
enc = tiktoken.get_encoding('gpt2')
print(enc.n_vocab)
print(enc.encode('hi there'))
print(enc.decode([5303, 612]))
# Just Offtopic: Review openai 

50257
[5303, 612]
hi there


In [8]:
# lets encode the entire text and inspect the first 1000 characters using torch
import torch
data = torch.tensor(encode(text), dtype=torch.int64)
print(data.shape, data.dtype)
print(data[:1000]) # first 1000 characters

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [9]:
# lets now split our dataset into training and validation sets
# we will use 90% of the data for training and 10% for validation
n = int(0.9 * len(data))
train_data, valid_data = data[:n], data[n:]

In [10]:
block_size = 8
train_data[:block_size +1]
# Karpathy talking about the 9 characters in the first block reminds me densenet paper about 
# particularly group of characters, 18 -> 47, 18,47->56, 18,47,56-> 57..  like here each character is connected 
# to all the previous characters in block, which is really like in densenet, 
# where each layer is connected to all the previous layers, which is really cool.

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is: {context} the target: {target}')

when input is: tensor([18]) the target: 47
when input is: tensor([18, 47]) the target: 56
when input is: tensor([18, 47, 56]) the target: 57
when input is: tensor([18, 47, 56, 57]) the target: 58
when input is: tensor([18, 47, 56, 57, 58]) the target: 1
when input is: tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is: tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is: tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [20]:
torch.manual_seed(1337)
batch_size = 4 # how many independent samples we want to process in parallel
block_size = 8 # what is maximum context length for the predictions

def get_batch(split):
    # generate a small batch of data inputs x and targets y
    data = train_data if split == 'train' else valid_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # starting index for each sequence
    x = torch.stack([data[i:i+block_size] for i in ix]) # input data
    # x_ = torch.stack([data[i:i+block_size] for i in ix]) 
    # print((x == x_).all())
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # target data
    return x, y

xb, yb = get_batch('train')
print('inputs:') 
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('-----------------')

for b in range(batch_size): # batch size
    for t in range(block_size): # sequence length
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'when input is: {context.tolist()} the target: {target}')

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----------------
when input is: [24] the target: 43
when input is: [24, 43] the target: 58
when input is: [24, 43, 58] the target: 5
when input is: [24, 43, 58, 5] the target: 57
when input is: [24, 43, 58, 5, 57] the target: 1
when input is: [24, 43, 58, 5, 57, 1] the target: 46
when input is: [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is: [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is: [44] the target: 53
when input is: [44, 53] the target: 56
when input is: [44, 53, 56] the target: 1
when input is: [44, 53, 56, 1] the target: 58
when input is: [44, 53, 56, 1, 58] the target

In [21]:
print(xb) # our input data for the transformer 

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [36]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C) - batch size, time steps, channels

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B* T, C)
            targets = targets.view(-1) # flatten the targets B*T
            loss = F.cross_entropy(logits, targets) # cross entropy needs input as (B, C, T) and target as (B, T), where T can be many dim, but C needs to be 2nd dim

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is a (B, T) tensor of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            id_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled token to the running sequence
            idx = torch.cat([idx, id_next], dim=1) # (B, T+1)
        return idx


m = BigramLanguageModel(vocab_size)
logits, loss =  m(xb, yb)
print(logits.shape)
print(loss)  # should be like ~ -ln(1/vocab_size) = -ln(1/65) = 4.17 if equal probability for all characters

idx = torch.zeros((1,1), dtype=torch.long)
print(repr(itos[0]))
print(decode(m.generate(idx, max_new_tokens=100).squeeze().tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)
'\n'

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [37]:
# create pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [44]:
batch_size = 32
for steps in range(10000):

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.450434446334839


In [47]:
print(decode(m.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=400).squeeze().tolist()))


IFLe:
Goweys wh ty beenn win my
To bee E dithourat
JUSCl oncll moue inaby, pe ls m t, iolit mase menof ll'd
Bor, arse tlle at tirnd, y we ty ay'te! toindilithespr alily well fthis I as r ovemyorerouthis matofrhe lavefessuladvoms thaven'd y Ind thig ies we:
Car Whan:
Habut.
Thont fo ind ove l tt:
TE llaclarr.
en th.
WAD win;
WI.
AMyothomo c
AUSI bend ant frrthi's buthareve y s avel,'t,
K: f-m roke 


### The mathematical trick in self attention

In [ ]:
# consider the following toy example

torch.manual_seed(1337)
B, T, C = 4,8,2 # batch, time, channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [49]:
# We want x[b,t] = mean_{i <= t} x[b,i]
xbow = torch.zeros((B, T, C)) # xbow - bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        xbow[b, t] = xprev.mean(dim=0) # (C)

In [56]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [58]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, dim=1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b 
print('a=')
print(a)
print('---')
print('b=')
print(b)
print('---')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
---
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
---
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
# version 2
wei = torch.tril(torch.ones(T,T))
wei = wei / torch.sum(wei, dim=1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)  so T,T -> B,T,T(adds B dim),then T,T @ T,C -> T,C then we get B,T,C
torch.allclose(xbow, xbow2)

True

In [ ]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [70]:
# version 4: Self-attention!
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # batch, time, channels
x = torch.randn(B, T, C)

# lets see a single head perform a self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, H) - batch, time, head
q = query(x) # (B, T, H) - batch, time, head
wei = q @ k.transpose(-2, -1) # (B, T, H) @ (B, H, T) -> (B, T, T)

tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x) # (B, T, H) - batch, time, head
out = wei @ v 
# out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [71]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [ ]:
k = torch.randn(B, T, head_size) 
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) / head_size ** 0.5 # divide by sqrt(d_k) - standard deviation 
# Var(cX) = c²Var(X) | Var(X + Y) = Var(X) + Var(Y) + 2Cov(X,Y) | Var(X + Y) = Var(X) + Var(Y) if X and Y are independent
# Var(X) = E[(X - E[X])²] = E[X²] - (E[X])² | E[cX] = cE[X] | 
# E stands for the expectation (or expected value) operator.  It represents the weighted average of 
# a random variable over its probability distribution.

In [76]:
print(k.var()) # unit gaussian with variance 1
print(q.var()) # unit gaussian with variance 1
print(wei.var()) # variance of dot product is 16 because of 16 head size and each pair priduct has variance 1

tensor(0.9006)
tensor(1.0037)
tensor(0.9957)


In [77]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [78]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1)

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

<img src="multi-head-attention.png" width="800">

In [ ]:
class LayerNorm1d:
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # layer mean, for each sample, mean of all features (dim= 0 is batch across all samples)
    xvar = x.var(1, keepdim=True) # layer variance, for each sample, variance of all features(dim= 0 is batch across all samples)
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]
  
torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32, 100 features
x = module(x)
x.shape

torch.Size([32, 100])

In [88]:
x[:,0].mean(), x[:,0].std() # mean, std of one of the features across all the batch inputs
# batch norm normalizes columns of the input data, so that each column has zero mean and unit variance

(tensor(0.1469), tensor(0.8803))

In [89]:
x[0,:].mean(), x[0,:].std() # mean, std of single input from the batch, of its features
# layer norm normalizes rows of the input data, so that each row has zero mean and unit variance

(tensor(2.3842e-09), tensor(1.0000))